# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [1]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col,count, desc
from pyspark.sql import Window
from pyspark.sql.functions import sum as Fsum

import datetime
import numpy as np
import pandas as pd


In [2]:
# create a Spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify subset") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [3]:
# read in sample data
df = spark.read.json("mini_sparkify_event_data.json")

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

### Gain understanding of dataset

We start by looking at some general dataset information to get a feel for what the dataset looks like

In [4]:
# investigate data schema
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [5]:
# look at a record
df.take(1)

[Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')]

In [6]:
# total number of records
df.count()

286500

In [7]:
# Distribution of pages visited
df.groupBy("page").count().collect()

[Row(page='Cancel', count=52),
 Row(page='Submit Downgrade', count=63),
 Row(page='Thumbs Down', count=2546),
 Row(page='Home', count=14457),
 Row(page='Downgrade', count=2055),
 Row(page='Roll Advert', count=3933),
 Row(page='Logout', count=3226),
 Row(page='Save Settings', count=310),
 Row(page='Cancellation Confirmation', count=52),
 Row(page='About', count=924),
 Row(page='Submit Registration', count=5),
 Row(page='Settings', count=1514),
 Row(page='Login', count=3241),
 Row(page='Register', count=18),
 Row(page='Add to Playlist', count=6526),
 Row(page='Add Friend', count=4277),
 Row(page='NextSong', count=228108),
 Row(page='Thumbs Up', count=12551),
 Row(page='Help', count=1726),
 Row(page='Upgrade', count=499),
 Row(page='Error', count=258),
 Row(page='Submit Upgrade', count=159)]

In [8]:
# total number of users
df.select("userId").dropDuplicates().count()

226

In [9]:
# Number of users going to each page
df.select("userId","page").dropDuplicates().groupBy("page").agg({"userID":"count"}).collect()

[Row(page='Cancel', count(userID)=52),
 Row(page='Submit Downgrade', count(userID)=49),
 Row(page='Thumbs Down', count(userID)=203),
 Row(page='Home', count(userID)=224),
 Row(page='Downgrade', count(userID)=154),
 Row(page='Roll Advert', count(userID)=207),
 Row(page='Logout', count(userID)=213),
 Row(page='Save Settings', count(userID)=132),
 Row(page='Cancellation Confirmation', count(userID)=52),
 Row(page='About', count(userID)=156),
 Row(page='Submit Registration', count(userID)=1),
 Row(page='Settings', count(userID)=195),
 Row(page='Login', count(userID)=1),
 Row(page='Register', count(userID)=1),
 Row(page='Add to Playlist', count(userID)=215),
 Row(page='Add Friend', count(userID)=206),
 Row(page='NextSong', count(userID)=225),
 Row(page='Thumbs Up', count(userID)=220),
 Row(page='Help', count(userID)=193),
 Row(page='Upgrade', count(userID)=168),
 Row(page='Error', count(userID)=118),
 Row(page='Submit Upgrade', count(userID)=131)]

In [10]:
# convert ts to months and days
get_month = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). month)
df = df.withColumn("month", get_month(df.ts))
get_day = udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0). day)
df = df.withColumn("day", get_day(df.ts))

In [11]:
# look at data time range
df.select('month','day').distinct().orderBy('month','day').collect()

[Row(month='10', day='1'),
 Row(month='10', day='10'),
 Row(month='10', day='11'),
 Row(month='10', day='12'),
 Row(month='10', day='13'),
 Row(month='10', day='14'),
 Row(month='10', day='15'),
 Row(month='10', day='16'),
 Row(month='10', day='17'),
 Row(month='10', day='18'),
 Row(month='10', day='19'),
 Row(month='10', day='2'),
 Row(month='10', day='20'),
 Row(month='10', day='21'),
 Row(month='10', day='22'),
 Row(month='10', day='23'),
 Row(month='10', day='24'),
 Row(month='10', day='25'),
 Row(month='10', day='26'),
 Row(month='10', day='27'),
 Row(month='10', day='28'),
 Row(month='10', day='29'),
 Row(month='10', day='3'),
 Row(month='10', day='30'),
 Row(month='10', day='31'),
 Row(month='10', day='4'),
 Row(month='10', day='5'),
 Row(month='10', day='6'),
 Row(month='10', day='7'),
 Row(month='10', day='8'),
 Row(month='10', day='9'),
 Row(month='11', day='1'),
 Row(month='11', day='10'),
 Row(month='11', day='11'),
 Row(month='11', day='12'),
 Row(month='11', day='13'),
 R

### Define Churn

The goal for this analysis is to understand user churn.  For this study churn means that a user has confirmed cancellation of thier account (either paid or free account).  To help with future work we will identify the rows where churn transaction occurs (actions with page equal to 'Cancellation Confirmation').  Additionally for users with a churn action we will tag all of their actions with identification that they are a user with churn.

In [12]:
# create column churn to flag churn events
flag_churn = udf(lambda x: 1 if x=="Cancellation Confirmation" else 0, IntegerType())
df = df.withColumn("churn",flag_churn("page"))
df.filter(df.churn==1).count() # confirm count matches number above

52

In [13]:
# create column churn_user to identify users with a churn event
windowval = Window.partitionBy("userId")
df = df.withColumn("churn_user", Fsum("churn").over(windowval))

In [14]:
# find some users with a churn action for testing
df.select("userId").filter(df.page == "Cancellation Confirmation").dropDuplicates().take(5)

[Row(userId='125'),
 Row(userId='51'),
 Row(userId='54'),
 Row(userId='100014'),
 Row(userId='101')]

In [15]:
# check columns for user 125
df.filter(df.userId=='125').select('ts','page','churn','churn_user').orderBy('ts').show()

+-------------+--------------------+-----+----------+
|           ts|                page|churn|churn_user|
+-------------+--------------------+-----+----------+
|1539317144000|            NextSong|    0|         1|
|1539317481000|            NextSong|    0|         1|
|1539317711000|            NextSong|    0|         1|
|1539317914000|            NextSong|    0|         1|
|1539318124000|            NextSong|    0|         1|
|1539318515000|            NextSong|    0|         1|
|1539318728000|            NextSong|    0|         1|
|1539318906000|            NextSong|    0|         1|
|1539318917000|         Roll Advert|    0|         1|
|1539318918000|              Cancel|    0|         1|
|1539318918000|Cancellation Conf...|    1|         1|
+-------------+--------------------+-----+----------+



### Exploratory Data Analysis

Now that we have churn identified we can start to explore the behavior of users that churn vs those that do not.  This exploration will establish some intuition regarding the data and help us prepare for feature engineering.

In [42]:
df.filter(df.userId=='51').select('ts','month','day','level','page').sort(desc("ts")).collect()

[Row(ts=1539761972000, month='10', day='17', level='paid', page='Cancellation Confirmation'),
 Row(ts=1539761830000, month='10', day='17', level='paid', page='Cancel'),
 Row(ts=1539761829000, month='10', day='17', level='paid', page='Settings'),
 Row(ts=1539761726000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539761508000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539761269000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539761030000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539760849000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539760549000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539760052000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539759811000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539759645000, month='10', day='17', level='paid', page='NextSong'),
 Row(ts=1539759362000, month='10', day='17', level='paid', pa

In [ ]:
# Number of users with free vs paid accounts
df.select("userId","page").dropDuplicates().groupBy("page").agg({"userID":"count"}).collect()

In [46]:
df.filter(df.userId=='51').select('ts','level','page').collect()

[Row(ts=1538398632000, level='paid', page='Home'),
 Row(ts=1538398649000, level='paid', page='NextSong'),
 Row(ts=1538398938000, level='paid', page='NextSong'),
 Row(ts=1538399167000, level='paid', page='NextSong'),
 Row(ts=1538399351000, level='paid', page='NextSong'),
 Row(ts=1538399495000, level='paid', page='NextSong'),
 Row(ts=1538399843000, level='paid', page='NextSong'),
 Row(ts=1538400086000, level='paid', page='NextSong'),
 Row(ts=1538400309000, level='paid', page='NextSong'),
 Row(ts=1538400621000, level='paid', page='NextSong'),
 Row(ts=1538400824000, level='paid', page='NextSong'),
 Row(ts=1538400891000, level='paid', page='Add to Playlist'),
 Row(ts=1538401249000, level='paid', page='NextSong'),
 Row(ts=1538401459000, level='paid', page='NextSong'),
 Row(ts=1538401460000, level='paid', page='Thumbs Down'),
 Row(ts=1538401688000, level='paid', page='NextSong'),
 Row(ts=1538401873000, level='paid', page='NextSong'),
 Row(ts=1538402170000, level='paid', page='NextSong'),
 Row

In [47]:
df.filter(df.userId=='51').select('ts','level','page').sort(desc("ts")).take(50)

[Row(ts=1539761972000, level='paid', page='Cancellation Confirmation'),
 Row(ts=1539761830000, level='paid', page='Cancel'),
 Row(ts=1539761829000, level='paid', page='Settings'),
 Row(ts=1539761726000, level='paid', page='NextSong'),
 Row(ts=1539761508000, level='paid', page='NextSong'),
 Row(ts=1539761269000, level='paid', page='NextSong'),
 Row(ts=1539761030000, level='paid', page='NextSong'),
 Row(ts=1539760849000, level='paid', page='NextSong'),
 Row(ts=1539760549000, level='paid', page='NextSong'),
 Row(ts=1539760052000, level='paid', page='NextSong'),
 Row(ts=1539759811000, level='paid', page='NextSong'),
 Row(ts=1539759645000, level='paid', page='NextSong'),
 Row(ts=1539759362000, level='paid', page='NextSong'),
 Row(ts=1539759213000, level='paid', page='Thumbs Up'),
 Row(ts=1539759212000, level='paid', page='NextSong'),
 Row(ts=1539759000000, level='paid', page='NextSong'),
 Row(ts=1539758750000, level='paid', page='NextSong'),
 Row(ts=1539758573000, level='paid', page='NextSo

In [48]:
df.filter(df.userId=='51').select('page').groupBy('page').agg({'page':'count'}).show()

+--------------------+-----------+
|                page|count(page)|
+--------------------+-----------+
|              Cancel|          1|
|         Thumbs Down|         21|
|                Home|         75|
|           Downgrade|         23|
|              Logout|         24|
|       Save Settings|          1|
|Cancellation Conf...|          1|
|               About|          2|
|            Settings|         12|
|     Add to Playlist|         52|
|          Add Friend|         28|
|            NextSong|       2111|
|           Thumbs Up|        100|
|                Help|         12|
|               Error|          1|
+--------------------+-----------+



In [49]:
df.filter(df.userId=='54').select('ts','level','page').collect()

[Row(ts=1538353930000, level='paid', page='NextSong'),
 Row(ts=1538354180000, level='paid', page='NextSong'),
 Row(ts=1538354396000, level='paid', page='NextSong'),
 Row(ts=1538354739000, level='paid', page='NextSong'),
 Row(ts=1538354749000, level='paid', page='Downgrade'),
 Row(ts=1538354985000, level='paid', page='NextSong'),
 Row(ts=1538355254000, level='paid', page='NextSong'),
 Row(ts=1538355255000, level='paid', page='Thumbs Up'),
 Row(ts=1538395957000, level='paid', page='NextSong'),
 Row(ts=1538396264000, level='paid', page='NextSong'),
 Row(ts=1538396638000, level='paid', page='NextSong'),
 Row(ts=1538396906000, level='paid', page='NextSong'),
 Row(ts=1538397069000, level='paid', page='NextSong'),
 Row(ts=1538397315000, level='paid', page='NextSong'),
 Row(ts=1538397475000, level='paid', page='NextSong'),
 Row(ts=1538397657000, level='paid', page='NextSong'),
 Row(ts=1538398061000, level='paid', page='NextSong'),
 Row(ts=1538398220000, level='paid', page='NextSong'),
 Row(ts=

In [50]:
df.filter(df.userId=='54').select('ts','level','page').sort(desc("ts")).take(50)

[Row(ts=1542051608000, level='paid', page='Cancellation Confirmation'),
 Row(ts=1542051577000, level='paid', page='Cancel'),
 Row(ts=1542051576000, level='paid', page='Settings'),
 Row(ts=1542051567000, level='paid', page='Home'),
 Row(ts=1542051505000, level='paid', page='NextSong'),
 Row(ts=1542051338000, level='paid', page='NextSong'),
 Row(ts=1542051079000, level='paid', page='NextSong'),
 Row(ts=1542050900000, level='paid', page='NextSong'),
 Row(ts=1542050648000, level='paid', page='NextSong'),
 Row(ts=1542050464000, level='paid', page='NextSong'),
 Row(ts=1542050203000, level='paid', page='NextSong'),
 Row(ts=1542049969000, level='paid', page='Home'),
 Row(ts=1542049964000, level='paid', page='NextSong'),
 Row(ts=1542049735000, level='paid', page='NextSong'),
 Row(ts=1542049586000, level='paid', page='Add to Playlist'),
 Row(ts=1542049505000, level='paid', page='NextSong'),
 Row(ts=1542049245000, level='paid', page='NextSong'),
 Row(ts=1542048907000, level='paid', page='NextSong

In [51]:
df.filter(df.userId=='54').select('page').groupBy('page').agg({'page':'count'}).show()

+--------------------+-----------+
|                page|count(page)|
+--------------------+-----------+
|              Cancel|          1|
|    Submit Downgrade|          1|
|         Thumbs Down|         29|
|                Home|        129|
|           Downgrade|         39|
|         Roll Advert|         47|
|              Logout|         36|
|       Save Settings|          3|
|Cancellation Conf...|          1|
|               About|          5|
|            Settings|         17|
|     Add to Playlist|         72|
|          Add Friend|         33|
|            NextSong|       2841|
|           Thumbs Up|        163|
|                Help|         17|
|             Upgrade|          1|
|               Error|          1|
|      Submit Upgrade|          1|
+--------------------+-----------+



In [54]:
df = df.withColumn("churn_id", Fsum("churn").over(windowval))

[Row(artist='Sleeping With Sirens', auth='Logged In', firstName='Darianna', gender='F', itemInSession=0, lastName='Carpenter', length=202.97098, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Captain Tyin Knots VS Mr Walkway (No Way)', status=200, ts=1539003534000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko) Version/7.0 Mobile/11D257 Safari/9537.53"', userId='100010', churn=0, churn_id=0),
 Row(artist='Francesca Battistelli', auth='Logged In', firstName='Darianna', gender='F', itemInSession=1, lastName='Carpenter', length=196.54485, level='free', location='Bridgeport-Stamford-Norwalk, CT', method='PUT', page='NextSong', registration=1538016340000, sessionId=31, song='Beautiful_ Beautiful (Album)', status=200, ts=1539003736000, userAgent='"Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like Mac OS X) AppleWebKit/537.51.2 (KHTML, like Gecko)

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.